# Lab 3 - Text Classification, NLTK, Naive Bayes

## Ревізії

In [3]:
import pandas as pd

revisions = pd.DataFrame(
    columns=["Date", "Comment"]
)
revisions = pd.concat([
    revisions,
    pd.DataFrame([{
        "Date": "Sep 20, 2019",
        "Comment": "Початкова версія"
    }])
], ignore_index=True)
revisions = pd.concat([
    revisions,
    pd.DataFrame([{
        "Date": "Oct 14, 2019",
        "Comment": "Виправлено орфографічні помилки"
    }])
], ignore_index=True)
revisions = pd.concat([
    revisions,
    pd.DataFrame([{
        "Date": "Oct 20, 2019",
        "Comment": "Додано сортування текстів в датасетах. Виправлено проблему TF/IDF = 0"
    }])
], ignore_index=True)
revisions

,Date,Comment
0,"Sep 20, 2019",Початкова версія
1,"Oct 14, 2019",Виправлено орфографічні помилки
2,"Oct 20, 2019",Додано сортування текстів в датасетах. Виправл...


## Мета

Використовуючи бібліотеку NLTK та алгоритм NaiveBayes побудувати модель визначення авторства текстів, написаних англійською мовою.

## Підготовка

Переконайтесь, що у вас встановлено пакети, необхідні для виконання роботи.

In [4]:
!pip install nltk
!pip install pandas
!pip install scikit-learn

Для обробки тексту ми використовуватимемо бібліотеку NLTK. Зробимо необхідні імпорти.

In [5]:
import nltk
import os
import glob
import math
import re
import pandas as pd
from pathlib import Path
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from matplotlib import pyplot as plt

Сконфігуруємо бібліотеки під потреби даної задачі.

In [6]:
# This is a directory where supporting files for NLTK library are located.
# This files include dictionaries for different languages, prebuilt models etc.
NLTK_DATA_DIR = os.path.expanduser("~/nltk_data")
nltk.data.path.insert(0, NLTK_DATA_DIR)

# Inlines Matplotlib graphs in Jupyter Notebook
%matplotlib inline

# Let's also enlarge the maximum width of the pandas column.
pd.set_option('display.max_colwidth', 150)

Для роботи NLTK необхідно завантажити деякі моделі та набори даних.

In [7]:
# Tokenizers
nltk.download("punkt", download_dir=NLTK_DATA_DIR)
nltk.download("wordnet", download_dir=NLTK_DATA_DIR)
nltk.download("stopwords", download_dir=NLTK_DATA_DIR)

[nltk_data] Downloading package punkt to C:\Users\Admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1 Завантаження даних та формування простору ознак

Разом з цією практичною ви завантажили набір творів англійською мовою. У вас є два набори даних. Перший навчальний (папка `train`), та другий тестовий (папка `test`). Усі твори погруповано по папках, де назва папки - це ім'я автора творів, що знаходяться в цій папці. Усі твори збережені в текстових файлах з розширенням файлу `txt`.

In [8]:
files = [(Path(f).parent.name, f) for f in glob.glob("train/**/*.txt", recursive=True)]
# Sort the files by relative file path
files.sort(key=lambda tup: tup[1])
for author, file in files:
    print(author, file)

### <span style="color:red">Завдання 1</span>

Завантажте дані з файлів та створіть `pandas.DataFrame` з колонками (`Author`, `Text`), де `Author` встановіть за назвою директорії, а `Text` - вміст файлу. Переконайтесь, що дані посортовано но відносному шляху до файлу (дивіться приклад вище).

In [9]:
def load_data(data_dir):
    data_dir_path = os.sep.join(['data', data_dir])
    authors_info = [dir_info[0].replace(os.sep.join([data_dir_path, ""]), '') for dir_info in os.walk(data_dir_path)][
                   1:]
    texts_info = [os.listdir(os.sep.join([data_dir_path, path, ""])) for path in authors_info]

    authors = []
    texts = []

    # Write your code here
    for author_texts in texts_info:
        author = authors_info[texts_info.index(author_texts)]

        authors += [author] * len(author_texts)
        texts += map(lambda path: Path(os.sep.join([data_dir_path, author, path])).read_text(), author_texts)
    # Make sure you have items sorted by relative file path

    return pd.DataFrame({'Author': authors, 'Text': texts})


def fix_data_types(df):
    df['Author'] = df.Author.astype(str)
    df['Text'] = df.Text.astype(str)
    return df


def load_train_data():
    return fix_data_types(load_data('train'))


def load_test_data():
    return fix_data_types(load_data('test'))

In [83]:
test_df = load_test_data()
train_df = load_train_data()
train_df.info()
train_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Author  2500 non-null   object
 1   Text    2500 non-null   object
dtypes: object(2)
memory usage: 39.2+ KB


,Author,Text
0,AaronPressman,The Internet may be overflowing with new technology but crime in cyberspace is still of the old-fashioned variety.\nThe National Consumers League ...
1,AaronPressman,The U.S. Postal Service announced Wednesday a plan to boost online commerce by enhancing the security and reliability of electronic mail traveling...
2,AaronPressman,"Elementary school students with access to the Internet learned more than kids who lacked access, an indepedent research group concluded after cond..."
3,AaronPressman,An influential Internet organisation has backed away from a proposal to dramatically expand the number of addresses available on the global comput...
4,AaronPressman,An influential Internet organisation has backed away from a proposal to dramatically expand the number of addresses available on the global comput...
5,AaronPressman,A group of leading trademark specialists plans to release recommendations aimed at minimizing disputes over Internet address names.\nThe Internati...
6,AaronPressman,"When a company in California sells a book to a consumer in Canada from a Web site hosted on a computer in England, what laws govern the transactio..."
7,AaronPressman,"U.S. laws governing the trillion dollar futures markets could be rocked by the Supreme Court's interpretation of the word ""in"" in a case to be arg..."
8,AaronPressman,Supreme Court justices Wednesday sharply questioned rules governing so-called derivative investments and foreign currency trading in a case that c...
9,AaronPressman,"The Internet continued to grow in leaps and bounds this year while online services found it much harder to add new customers, a survey said Friday..."


### <span style="color:red">Завдання 2</span>

Обчисліть кількість творів для кожного автора в навчальному датасеті.

In [11]:
# Write your code here
train_df.groupby("Author").count()

,Text
Author,
AaronPressman,50
AlanCrosby,50
AlexanderSmith,50
BenjaminKangLim,50
BernardHickey,50
BradDorfman,50
DarrenSchuettler,50
DavidLawder,50
EdnaFernandes,50


**Визначення**. Датасет називається `збалансованим` якщо кількість запитів в для кожної мітки є приблизно однаковою.

**Запитання**. Враховуючи що Автор в даному випадку є міткою датасету, дайте відповідь на запитання чи є навчальний датасет збалансованим?

<span style="color:red">**Ваша відповідь**:</span> Датасет не є збалансованим, оскільки на 1 автора (мітка x) приходиться n (n > 1) творів (мітка y)

**Зауваження**. Якщо датасет збалансований, ви можете використовувати практично будь-який алгоритм машинного навчання. Якщо датасет розбалансований, вам необхідно його додатково обробляти (збалансувати), або використовувати алгоритми, що вміють працювати з незбалансованими датасетами.

### <span style="color:red">Завдання 3</span>

<span style="color:red">**a)**</span> Об'єднайте всі тексти з `train_df` у одну стрічку. Назвіть стрічку `combined_text`.

In [12]:
combined_text = ' '.join(list(train_df['Text']))

<span style="color:red">**b)**</span> Напишіть функцію `normalize_text` та замініть у тексті знаки нового рядка (`\n`) на пропуски (` `). Якщо ви вважаєте, що реєстр літер не важливий для даної задачі, то приведіть усі символи до одного реєстру. Також на власний розсуд замініть інші неважливі знаки у тексті.

In [13]:
def normalize_text(text):
    """
    Normalizes the input string for the task of authorship verification
    """
    normalized_text = re.sub(r'\d+|\\p{Punct}', '', text.replace('\n', ' ').lower()).strip()
    # Write your code here

    return normalized_text

<span style="color:red">**c)**</span> Використовуючи NLTK, напишіть функцію `tokenize_text`, яка поділить текст на окремі слова.

In [14]:
def tokenize_text(text):
    """
    Splits the input string into a list of lexical units
    """
    tokenized = nltk.word_tokenize(text)
    # Write your code here
    return tokenized

Створюємо `Series` з слів тексту.

In [15]:
words = pd.Series(tokenize_text(normalize_text(combined_text)))
words

0                  the
1             internet
2                  may
3                   be
4          overflowing
              ...     
1416404            for
1416405            the
1416406       congress
1416407              .
1416408             ''
Length: 1416409, dtype: object

### <span style="color:red">Завдання 4</span>

<span style="color:red">**a)**</span> Скільки всього унікальних слів в `words`?

In [16]:
unique_words_count = pd.Series(words).nunique()  # Write your code here
unique_words_count

31343

<span style="color:red">**b)**</span> Виведіть 30 найбільш вживаних слів? Як ви вважаєте, чи важливі ці слова при визначенні авторства? Чому?

No influence is encountered cause these words are used in general grammatical constructions

In [17]:
# write your code here
word_counts = pd.Series(words).value_counts()
word_counts[:30]

the     71352
,       63452
.       60300
to      36280
of      33921
a       29140
in      27699
and     25928
said    19856
's      14877
``      13847
''      13747
for     12249
on      12085
that    10478
is       9923
it       9536
with     7794
be       7518
$        7466
at       7283
by       7215
its      6888
as       6874
was      6710
from     6178
he       5976
will     5877
but      5622
has      5560
Name: count, dtype: int64

## TF-IDF

При аналізі тексту є ряд ключових показників оцінки важливості слів. Розглянемо їх в цьому розділі.

Частота слова (Term Frequency або скорочено TF) - відношення числа входжень обраного слова до загальної кількості слів документу.

\begin{align}
df = \frac{number\;of\;instances\;of\;the\;word\;in\;the\;document}{total\;number\;of\;words\;in\;document}
\end{align}

### <span style="color:red">Завдання 5</span>

Обчисліть TF для слова `internet` в першому творі навчального датасету.

In [30]:
def word_tf(word):
    return word_counts[word] / word_counts.sum()

In [31]:
tf = word_tf('internet')
# Write your code here
print("Term frequency for word 'internet' is", tf)

Term frequency for word 'internet' is 0.0006262315475261736


Обернена частота документа (Inverse Document Frequency або скорочено IDF) - інверсія частоти, з якою слово зустрічається в документах колекції. Чим частіше слово зустрічається у всіх документах колекції тим нижчим буде його IDF.

\begin{align}
idf = \log(\frac{number\;of\;documents\;in\;the\;dataset}{number\;of\;documents\;in\;the\;dataset\;which\;contain\;given\;word})
\end{align}

Зверніть увагу, що основа логарифму не має значення.

### <span style="color:red">Завдання 6</span>

Обчисліть IDF для слів `deposit` та `business`. Використайте натуральний логарифм (основа `e`).

In [32]:
def word_idf(word):
    return math.log(len(train_df['Text']) / len(train_df[train_df['Text'].str.contains(word)]['Text']))

In [33]:

idf_deposit = word_idf('deposit')
idf_business = word_idf('business')
# Write your code here
print("Term frequency for work 'deposit' is", idf_deposit)
print("Term frequency for work 'business' is", idf_business)

Term frequency for work 'deposit' is 2.995732273553991
Term frequency for work 'business' is 0.906340401020987


TF-IDF визначається як добуток двох множників: TF та IDF. Таким чином слова з високою частотою появи в межах документа та низькою частотою вживання в інших документах отримують більшу вагу.

### <span style="color:red">Завдання 7</span>

Обчисліть TF-IDF для слів `scam` та `money` в першому документі навчального датасету.

In [34]:
def word_tf_idf(word):
    return word_tf(word) * word_idf(word)

In [35]:
# Write your code here
scam_tf_idf = word_tf_idf('scam')
money_tf_idf = word_tf_idf('money')

scam_tf_idf, money_tf_idf

(3.178077198617107e-05, 0.0006946770858845027)

## 2 Побудова простої моделі

На щастя, витягнення TF-IDF вже імплементовано за вас у бібліотеці `scikit-learn`. Для цього існує клас [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

**Зауваження.** Практично всі моделі в Scikit-Learn мають спільний інтуітивний інтерфейс. В конструкторі моделі ви передаєте набір параметрів для конфігурації моделі.Є дві групи моделей: моделі обробки та моделі класифікації.

Моделі обробки даних зазвичай мають методи `fit_transform` та `transform`. Метод `fit_transform` використовується одночасно для навчання моделі на даних та їх перетворення, а метод `transform` виключно для перетворення даних (з використанням попередньо навченої моделі).

Моделі класифікації даних зазвичай мають методи `fit` та `predict`. Метод `fit` передбачає навчання моделі, а метод `predict` - використання вже навченої моделі для отримання передбачень.

**Важливо**. Використовуйте методи `fit` та `fit_transform` виключно з навчальними даними. Для тестових даних використовуйте `tranform` та `predict`.

### <span style="color:red">Завдання 8</span>

Створіть об'єкт класу TfidfVectorizer. Сконфігуруйте його з наступними параметрами:
- sublinear_tf=True
- max_df=0.5
- stopwords='english'

Виконайте навчання та перетворення документів з навчального та тестового датасетів.
Результат перетворення документів навчального датасету запишіть у `features_train_transformed`. Результат перетворення документів тестового датасету запишіть у `features_test_transformed`.

In [49]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    max_df=.5,
    stop_words='english'
)

features_train_transformed = vectorizer.fit_transform(train_df['Text'])
features_test_transformed = vectorizer.transform(test_df['Text'])

Який розмір має матриця `features_train_transformed`?

In [53]:
features_train_transformed.shape, features_test_transformed.shape

((2500, 28839), (2500, 28839))

Як ви помітили, матриця має доволі багато характеристик документу. Це може спричинити проблеми при навчанні, тому давайте усунемо ряд характеристик, що мають найменший вплив на модель.

### <span style="color:red">Завдання 9</span>

Використайте [`SelectPerceptile`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html) з функцією оцінки впливу `f_classif`. Встановіть значення `percentile` в 20%.

Результат перетворення `features_train_transformed` запишіть у `features_train_transformed_optimized`. Результат перетворення `features_test_transformed` запишіть у `features_test_transformed_optimized`.

In [57]:
percentileTransformer = SelectPercentile(f_classif, percentile=20)
features_train_transformed_optimized = percentileTransformer.fit_transform(features_train_transformed,
                                                                           train_df['Author']).toarray()
features_test_transformed_optimized = percentileTransformer.transform(features_test_transformed).toarray()


Який розмір має матриця `features_train_transformed_optimized`?

In [58]:
features_train_transformed_optimized.shape, features_test_transformed_optimized.shape

((2500, 5768), (2500, 5768))

На основі отриманих характеристик будуємо модель. Використайте код нижче. Поки що ми не будемо вдаватись в деталі реалізації моделі.

In [59]:
clf = GaussianNB()
clf.fit(features_train_transformed_optimized, train_df['Author'])
predicted = clf.predict(features_test_transformed_optimized)
predicted

array(['AaronPressman', 'AaronPressman', 'TheresePoletti', ...,
       'WilliamKazer', 'GrahamEarnshaw', 'BenjaminKangLim'], dtype='<U17')

## 3 Оцінка моделей

Розглянемо ситуацію коли ми маємо дві моделі, які виконують однакову роботу (передбачають автора). Як порівняти ці моделі? Яка працює краще, а яка гірше? Відповідь на ці запитання можна отримати за допомогою методів оцінки моделей.

Одним з базових методів оцінки моделі є точність (`accuracy`). Точність визначається кількість правильно класифікованих елементів певного датасету, до загальної кількості елементів в цьому датасеті.

In [60]:
correct_labels = test_df['Author']
accuracy_score(correct_labels, predicted)

0.5816

Іншими методами оцінки якості моделі є влучність (precision) та повнота (recall).

![](https://miro.medium.com/max/1872/1*pOtBHai4jFd-ujaNXPilRg.png)

In [61]:
precision_score(correct_labels, predicted, average='weighted')

0.6225548364039921

In [62]:
recall_score(correct_labels, predicted, average='weighted')

0.5816

### <span style="color:red">Завдання 10</span>

Встановлюючи значення 10, 30, 40, 50 по черзі у поле `percentile` в `SelectPercentile` перегенеруйте навчальні та тестові дані. На основі нових даних виконайте навчання моделі та оцініть її якість. Як змінюється точність моделі зі зміною значення `perceptile` (більшою кількістю характеристик).

In [63]:
def get_gauss_nb(X, y, X_test):
    clf = GaussianNB()
    clf.fit(X, y)
    predicted = clf.predict(X_test)

    return predicted


In [67]:
for percentile in (10, 30, 40, 50):
    percentile_transformer = SelectPercentile(f_classif, percentile=percentile)

    features_train_transformed_optimized = percentile_transformer.fit_transform(features_train_transformed,
                                                                                train_df['Author']).toarray()
    features_test_transformed_optimized = percentile_transformer.transform(features_test_transformed).toarray()

    prediction = get_gauss_nb(features_train_transformed_optimized, train_df['Author'],
                              features_test_transformed_optimized)
    precision, recall = (precision_score(correct_labels, prediction, average='weighted'),
                         recall_score(correct_labels, prediction, average='weighted'))
    print(precision, recall)


0.6102268932985803 0.5772
0.6276242141550642 0.5844
0.6377922887410735 0.5852
0.6433692749714219 0.594


## Додаткові методи обробки тексту  (на максимальну кількість балів)

### <span style="color:red">Завдання 11</span>

Використовуючи функцію `normalize_text` та `tokenize_text`, перетворіть колонку `Text` в навчальному та тестовому датасетах з стрічки у масив слів.

In [82]:
def expand_contractions(sentence: str):
    contractions_dict = {
        "ain’t": "are not",
        "’s": " is",
        "aren’t": "are not",
        "can’t": "cannot",
        "can’t’ve": "cannot have",
        "‘cause": "because",
        "could’ve": "could have",
        "couldn’t": "could not",
        "couldn’t’ve": "could not have",
        "didn’t": "did not",
        "doesn’t": "does not",
        "don’t": "do not",
        "hadn’t": "had not",
        "hadn’t’ve": "had not have",
        "hasn’t": "has not",
        "haven’t": "have not",
        "he’d": "he would",
        "he’d’ve": "he would have",
        "he’ll": "he will",
        "he’ll’ve": "he will have",
        "how’d": "how did",
        "how’d’y": "how do you",
        "how’ll": "how will",
        "I’d": "I would",
        "I’d’ve": "I would have",
        "I’ll": "I will",
        "I’ll’ve": "I will have",
        "I’m": "I am",
        "I’ve": "I have",
        "isn’t": "is not",
        "it’d": "it would",
        "it’d’ve": "it would have",
        "it’ll": "it will",
        "it’ll’ve": "it will have",
        "let’s": "let us",
        "ma’am": "madam",
        "mayn’t": "may not",
        "might’ve": "might have",
        "mightn’t": "might not",
        "mightn’t’ve": "might not have",
        "must’ve": "must have",
        "mustn’t": "must not",
        "mustn’t’ve": "must not have",
        "needn’t": "need not",
        "needn’t’ve": "need not have",
        "o’clock": "of the clock",
        "oughtn’t": "ought not",
        "oughtn’t’ve": "ought not have",
        "shan’t": "shall not",
        "sha’n’t": "shall not",
        "shan’t’ve": "shall not have",
        "she’d": "she would",
        "she’d’ve": "she would have",
        "she’ll": "she will",
        "she’ll’ve": "she will have",
        "should’ve": "should have",
        "shouldn’t": "should not",
        "shouldn’t’ve": "should not have",
        "so’ve": "so have",
        "that’d": "that would",
        "that’d’ve": "that would have",
        "there’d": "there would",
        "there’d’ve": "there would have",
        "they’d": "they would",
        "they’d’ve": "they would have",
        "they’ll": "they will",
        "they’ll’ve": "they will have",
        "they’re": "they are",
        "they’ve": "they have",
        "to’ve": "to have",
        "wasn’t": "was not",
        "we’d": "we would",
        "we’d’ve": "we would have",
        "we’ll": "we will",
        "we’ll’ve": "we will have",
        "we’re": "we are",
        "we’ve": "we have",
        "weren’t": "were not",
        "what’ll": "what will",
        "what’ll’ve": "what will have",
        "what’re": "what are",
        "what’ve": "what have",
        "when’ve": "when have",
        "where’d": "where did",
        "where’ve": "where have",
        "who’ll": "who will",
        "who’ll’ve": "who will have",
        "who’ve": "who have",
        "why’ve": "why have",
        "will’ve": "will have",
        "won’t": "will not",
        "won’t’ve": "will not have",
        "would’ve": "would have",
        "wouldn’t": "would not",
        "wouldn’t’ve": "would not have",
        "y’all": "you all",
        "y’all’d": "you all would",
        "y’all’d’ve": "you all would have",
        "y’all’re": "you all are",
        "y’all’ve": "you all have",
        "you’d": "you would",
        "you’d’ve": "you would have",
        "you’ll": "you will",
        "you’ll’ve": "you will have",
        "you’re": "you are",
        "you’ve": "you have"
    }

    contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
    return contractions_re.sub(lambda match: contractions_dict[match.group(0)], sentence)


In [84]:
test_df = load_test_data()
train_df = load_train_data()

text_train_contracted = train_df["Text"].map(expand_contractions)
text_test_contracted = test_df["Text"].map(expand_contractions)

train_df["Text"] = text_train_contracted.map(lambda s: word_tokenize(s, 'english'))
test_df["Text"] = text_test_contracted.map(lambda s: word_tokenize(s, 'english'))

train_df["Text"], test_df["Text"]

(0       [The, Internet, may, be, overflowing, with, new, technology, but, crime, in, cyberspace, is, still, of, the, old-fashioned, variety, ., The, Natio...
 1       [The, U.S, ., Postal, Service, announced, Wednesday, a, plan, to, boost, online, commerce, by, enhancing, the, security, and, reliability, of, ele...
 2       [Elementary, school, students, with, access, to, the, Internet, learned, more, than, kids, who, lacked, access, ,, an, indepedent, research, group...
 3       [An, influential, Internet, organisation, has, backed, away, from, a, proposal, to, dramatically, expand, the, number, of, addresses, available, o...
 4       [An, influential, Internet, organisation, has, backed, away, from, a, proposal, to, dramatically, expand, the, number, of, addresses, available, o...
                                                                                 ...                                                                          
 2495    [China, 's, central, bank, chief, has

## Шумові слова (Stopwords)

Шумові слова або стоп-слова - це слова, які не несуть смислового навантаження, тому їх користь та роль не є суттєвою. У кожної мови є свої набори слів, які практично завжди можна прибрати.

Для англійської мови це наприклад:
- артиклі (a, an, the)
- сполучні слова (and, or)
- займенники (I, You, We, They)
- слова для позначення часу (is, are, be, was, were, will, have, has etc)
- числа
- інші частовживані слова

Додатково можна видалити усі числа, окремо розташовані знаки пунктуації `. , = + /! "; :%? * ()`.

Також можна видаляти найбільш частовживані слова предметної області документів (якщо актуально)

В NLTK вже є готові побудовані набори найбільш частовживаних слів англійської мови, доступні через клас stopwords. 

### <span style="color:red">Завдання 12</span>

<span style="color:red">**a)**</span> Виведіть стоп-слова для англійської мови з пакету NLTK. Скільки всього слів?

In [85]:
stop_words = stopwords.words()

<span style="color:red">**b)**</span> Видаліть з масивів слів у навчальному та тестовому датафреймах стоп-слова.

In [ ]:
func = lambda x: [word for word in x if word not in stop_words]
train_df['Text'] = train_df['Text'].apply(func)
test_df['Text'] = test_df['Text'].apply(func)

<span style="color:red">**c)**</span> Видаліть з масивів слів у навчальному та тестовому датасетах окремі знаки пунктуації та числа.

In [ ]:
nltk.download("punkt")

func = lambda x: [word for word in x if word.isalpha()]
train_df['Text'] = train_df['Text'].apply(func)
test_df['Text'] = test_df['Text'].apply(func)

##  Нормалізація слів

Під початковою формою слова мається на увазі форма слова, яка не зазнала змін, необзідних для передачі граматичних категорій, таких як час, кількість, особа, стать, тощо.

Наприклад, слово `бути` в залежності від часу та особи може записуватись як `був`, `була`, `були`, `є`, `буду`, `будуть`, `будете` тощо.

Процес приведення слова до початкової форми називаються `Лемматизацією` (Lemmatization).

In [ ]:
wnl = WordNetLemmatizer()
print(wnl.lemmatize('getting', 'v'))
print(wnl.lemmatize('rabbits', 'n'))
print(wnl.lemmatize('quickly', 'r'))
print(wnl.lemmatize('slowly', 'r'))
# KeyError! - Doesn't work on non-words!
print(wnl.lemmatize('xyzing', ''))

Проте, лемматизації вимагає правильно визначити частину мови, що є нетривіальною задачею.

Також інколи потрібно розглядати слова не в структурі частин мови, а за коренем. Наприклад, слова `Organization` та `To Organize` з точки зору лемматизації є різними, хоча корінь в них спільний. І якщо ви часто використовуєте одне з слів, то швидше за все будете використовувати і інше.

Тож часто задачі аналізу тексту використовують не лемматизацію, а стеммінг (Stemming). Процес стеммінгу полягає у знаходження коренів слів (не обов'язково лінгвістично правильних).

Наприклад, під час стеммінгу слова `organized`, `organization`, `organizers` можуть бути приведені до єдиного слова `organiz`.

NLTK містить набір декількох стандартних стеммерів. Ознайомтесь з ними нижче.
Ви можете використовувати також сторонні стеммери наприклад, https://pypi.org/project/stemming/1.0/

In [ ]:
stemmers = [PorterStemmer(), SnowballStemmer("english"), LancasterStemmer()]
sentence = "Stemming and Lemmatization are Text Normalization (or sometimes called Word Normalization) techniques in the field of Natural Language Processing that are used to prepare text, words, and documents for further processing. Stemming and Lemmatization have been studied, and algorithms have been developed in Computer Science since the 1960's."
sentence = tokenize_text(sentence)
print("Original sentence:", ' '.join(sentence))
for stemmer in stemmers:
    print(type(stemmer).__name__ + ":", ' '.join([stemmer.stem(w) for w in sentence]))

### <span style="color:red">Завдання 13</span>

Прочитайте детальніше документацію NLTK щодо `PorterStemmer` та `SnowballStemmer`. В чому полягає різниця між ними? Напишіть речення, для на якому вивід `PorterStemmer` та `SnowballStemmer` будуть різним.

In [ ]:
# Write your code here

### <span style="color:red">Завдання 14</span>

Використайте один із стеммерів (на ваш розсуд) для обробки слів в навчальному та тестовому датасетах.

In [ ]:
def normalize_words(words):
    # Write your code here
    return words

In [ ]:
train_df['Text'] = train_df['Text'].apply(lambda x: normalize_words(x))
train_df

### <span style="color:red">Завдання 15 </span>

На основі оброблених слів перебудуйте модель NaiveBayes, створену вище та проведіть її оцінку. Чи змінилась точність?

In [ ]:
# Write your code here